## create mfe GP map for RNA12


In [3]:
#all of these will be useful for the following code
import numpy as np
import itertools as it
from collections import defaultdict
from collections import Counter
import RNA #careful with cluster, should have installed ViennaRNA
import pickle
import math
import os 
import sys

L=12 #length of RNA is 12 

In [5]:
#Generate all RNA sequences of L=12

components = ['A','U','G','C']
listRNA = [list(sequence) for sequence in it.product(components, repeat=L)]
with open("seqRNA12fold.txt", "a") as output:
    for i in range(0,len(listRNA)):
        for j in np.arange(0,len(listRNA[0])):
            output.write(str(listRNA[i][j]))
        output.write('\n')

In [10]:
def create_RNA_D_GPmap(filename,L):
    count = 0
    with open(filename) as fp:
        d = defaultdict()
        while True:
            count += 1
            line = fp.readline()
 
            if not line:
                break
            if count%2 != 0:
                seq = line.strip()
            else:
                fold = line.strip()
                d[seq[0:L]] = fold[0:L]
                        
    return d

In [ ]:
# Fold them in termial:
# RNAfold < seqRNA12fold.txt > output.txt

In [7]:
filename = "output.txt"
#get MFE GP map
ddictRNA12 = create_RNA_D_GPmap(filename,L)

#get MFE neutralsets 
dneutralsetRNA12 = Counter(ddictRNA12.values())

## create non-deterministic GP map for RNA12


We use energy gap of 15kbT

In [11]:
def normalize(probs):
    prob_factor = 1 / sum(probs)
    return [prob_factor * p for p in probs]

In [1]:
kbT_RNA = RNA.exp_param().kT/1000.0
cutoff = 15*kbT_RNA
print(kbT_RNA,cutoff)

0.6163207755 9.2448116325


1.  First in terminal (we do mfe + [15*kbT_RNA=9.2448116325] cutoff): 

    RNAsubopt -e 9.2448116325 < seqRNA12fold.txt > subopt12tot.txt

    RNAsubopt -p < seqRNA12fold.txt > foldF_RNA12.txt
    
    RNAfold < seqRNA12fold.txt > folds12.txt
2. Run the code below
3. Save dictRNA12

In [ ]:

dictRNA12 = defaultdict(list)
dictfolds12 = {}
dictRNA12F = {}
count = 0
with open('folds12.txt','r') as handle:
    for line in handle:
        count+=1
        if count%2!=0:
            seq = line
        elif line[0:L] == '.'*L:
            dictfolds12[seq] = '.'*L+ ' '*2 + str(0.0)
            continue
        else: 
            dictfolds12[seq] = line
with open ('foldF_RNA12.txt','r') as f:
    for line in f:
        if line in dictfolds12.keys():
            seq = line[0:L]
            continue
        elif line[L+1]=='[':
            dictRNA12F[seq] = line[L+3:-2]
            continue
probs = []
folds = []
count = 0

with open('subopt12tot.txt','r') as ff:
    for line in ff:

        if line[0:L]+'\n' in dictfolds12.keys():
            count+=1
            if  count!=1:
                if '.'*L not in folds:
                    probunfolded = math.exp(float(dictRNA12F[seq])/kbT_RNA)
                    probs.append(probunfolded)
                    folds.append('.'*L)

                seqProbs = normalize(probs)
                for f,p in zip(folds,seqProbs):
                    dictRNA12[seq].append(f+ ' '*2 + str(p))

            seq = line[0:L]
            folds = []
            probs = []
            continue


        else:
        
            folds.append(line[0:L])
            prob = math.exp((float(dictRNA12F[seq])-float(line[L+2:-1]))/kbT_RNA)
            probs.append(prob)
            
            continue
       
if '.'*L not in folds:
    probunfolded = math.exp(float(dictRNA12F[seq])/kbT_RNA)
    probs.append(probunfolded)
    folds.append('.'*L)

seqProbs = normalize(probs)
for f,p in zip(folds,seqProbs):
    dictRNA12[seq].append(f+ ' '*2 + str(p))

# Realisations of ND GP map
Once dictionary of ND GP map (heavy file) is saved, we can create a sample (size 500) of realisations.

In [ ]:
#Use the code below to create each of the 500 realisations from total ND GP map

def normalize(probs):
    prob_factor = 1 / sum(probs)
    return [prob_factor * p for p in probs]
def extractnormalisedprobs(pboltzlist,L):
    probsnorm = []
    for p in pboltzlist:
        probsnorm.append(float(p[L+2:]))
    prob = normalize(probsnorm)
    return {pboltzlist[pi][0:L]: prob[pi] for pi in range(0,len(pboltzlist))}


def sampleGP(dictsuboptRNA12,L):
    realisationgp = {}
    for seq,subopt in dictsuboptRNA12.items():
        foldList = []
        probsList = []
        phvsprobseq = extractnormalisedprobs(dictsuboptRNA12[seq],L)
        for phenotype,probg in phvsprobseq.items():
            foldList.append(phenotype)
            probsList.append(probg)
        realisationgp[seq] = random.choices(foldList, weights = probsList, k=1)[0]
    return realisationgp

# Deterministic Quantities (neutral sets, robustness, evolvability)

We have a dictionary that we will call DGPmap representing the deterministic GP map of RNA12

In [ ]:
#useful in functions of quantities
def normalize(probs):
    prob_factor = 1 / sum(probs)
    return [prob_factor * p for p in probs]

def mutationalneighbours(seq):
    mutations = {'A': ['C','U','G'],'C': ['A','U','G'],'G': ['A','U','C'], 'U':['A','G','C']}
    return [seq[:j] + m + seq[j+1:] for j in range(0,len(seq)) for m in mutations[str(seq[j])]]

## 1. Neutral sets

In [ ]:
#dictionary of neutral set sizes. Each fold has a value representing the number of times it appears in GP map dict
neutralsetsD = Counter(list(DGPmap.values()))

## 2. Robustness

In [ ]:
#folds is an array of all the total folds of the GP map (same size as genotype space)
def robustnessD(DGPmap,folds,K,L):   
    s = list(DGPmap.keys())
    a = Counter(folds)
    rho_g = {}
    for A in DGPmap.keys():
        rho_g[A[0:L]] = 0
    rho_p = {}
    for A in a.keys():
        rho_p[A[0:L]] = 0
    #genotypic robustness
    for seq in DGPmap.keys():
        for mut in mutationalneighbours(seq):
                if (DGPmap[mut]==DGPmap[seq]):rho_g[seq]+=1/((K-1)*L)
                else: continue
    #phenotypic robustness
    for fold,count in a.items():
        for j in range(0,len(folds)):
            if fold[0:L]==folds[j][0:L]:
                rho_p[fold[0:L]]+=rho_g[s[j]]
        rho_p[fold[0:L]]/= count
    
    return rho_g,rho_p

## 3. Evolvability

In [ ]:
#genotypic evolvability
def evolvabilitygD(DGPmap):
    evgdictd = defaultdict(float)
    for seq,p in DGPmap.items():
            prodfold = defaultdict(lambda:1)
            for newmutation in mutationalneighbours(seq):            
                    foldmut=DGPmap[newmutation]
                    if foldmut != p:
                        prodfold[foldmut] *=0
                    else: prodfold[foldmut]*=1
            for pprime in prodfold.values():
                evgdictd[seq]+=(1-pprime)
    return evgdictd

In [ ]:
#phenotypic evolvability
def evolvabilitypD(DGPmap):
    evolp = defaultdict(float)
    prodfold = defaultdict(lambda:1) 
    for seq, phenotype in DGPmap.items():
        for newmutation in mutationalneighbours(seq):                    
            phenotypeprime = DGPmap[newmutation]
            if phenotypeprime!= phenotype:
                prodfold[(phenotype, phenotypeprime)] *=0

    for (phenotype, phenotypeprime), val in prodfold.items():
        evolp[phenotype] += (1-val)

    return evolp

# Non-Deterministic Quantities (neutral sets, robustness, evolvability)

We have a dictionary that we will call NDGPmap representing the non-deterministic GP map of RNA12

In [ ]:
def normalize(probs):
    prob_factor = 1 / sum(probs)
    return [prob_factor * p for p in probs]

def mutationalneighbours(seq):
    mutations = {'A': ['C','U','G'],'C': ['A','U','G'],'G': ['A','U','C'], 'U':['A','G','C']}
    return [seq[:j] + m + seq[j+1:] for j in range(0,len(seq)) for m in mutations[str(seq[j])]]

#For ND quant: normalise probabilities and return resulting dict of the boltzmann folds ens and probs.
def extractnormalisedprobs(pboltzlist,L):
    probsnorm = []
    for p in pboltzlist:
        probsnorm.append(float(p[L+2:]))
    prob = normalize(probsnorm)
    return {pboltzlist[pi][0:L]: prob[pi] for pi in range(0,len(pboltzlist))}

## 1. Neutral sets

In [ ]:
def neutralsetsND(NDGPmap,K,L):
    
    folddict = defaultdict(list)
    folds = []
    NDsetsize = defaultdict(float)
    
    for seq in NDGPmap.keys():
        phvsprobseq = extractnormalisedprobs(NDGPmap[seq],L)
        for phenotype,probg in phvsprobseq.items():
            folds.append(phenotype) # total list of folds in dict
            NDsetsize[phenotype] += probg #dictionary of neutral set sizes
            folddict[phenotype].append([seq,probg]) # folds dict that gives each seq and prob where it appears
            
    return NDsetsize,folddict,folds

## 2. Robustness

In [ ]:
#genotypic robustness, for each genotype
def robustnessgND(NDGPmap,seq,K,L):    
    rhogndict = defaultdict(float)
    phvsprobseq = extractnormalisedprobs(NDGPmap[seq],L)
    neighbourvsphvsprob = {newmutation: extractnormalisedprobs(NDGPmap[newmutation],L) for newmutation in mutationalneighbours(seq)}
    for phenotype, probg in phvsprobseq.items():
        for newmutation, phvsprobmut in neighbourvsphvsprob.items():
                try:
                    probpgmut = phvsprobmut[phenotype] 
                    rhogndict[seq] += probpgmut*probg/((K-1)*L) 
                except KeyError:
                    continue     
    del neighbourvsphvsprob
    return rhogndict

In [ ]:
#phenotypic robustness
def robustnesspND(folddict,NDGPmap):
    rhopndict = defaultdict(float)
    for f,seqprobs in folddict.items():
        probgtot = 0
        for seq_p in seqprobs:
            seq=seq_p[0]
            probg =float(seq_p[1])
            probgtot += float(seq_p[1])
            neighbourvsphvsprob = {newmutation: extractnormalisedprobs(NDGPmap[newmutation],L) for newmutation in mutationalneighbours(seq)}
            for newmutation, phvsprobmut in neighbourvsphvsprob.items():
                for phenomut,probpgmut in phvsprobmut.items():
                    if f == phenomut:
                        rhopndict[f] += probpgmut*probg
            del neighbourvsphvsprob

        rhopndict[f]/=probgtot
    return rhopndict

## 3. Evolvability

In [ ]:
#genotypic evolvability, for each genotype
def evolvabilitygND(gpmap,seq,K,L):
    evgndict = defaultdict(float)
    phvsprobseq = extractnormalisedprobs(gpmap[seq],L)
    neighbourvsphvsprob = {newmutation: extractnormalisedprobs(gpmap[newmutation],L) for newmutation in mutationalneighbours(seq)}
    for phenotype, probg in phvsprobseq.items():
        probfold = defaultdict(lambda:1)
        evgndictp = 0
        for newmutation, phvsprobmut in neighbourvsphvsprob.items():
            for phenomut,probgmut in phvsprobmut.items():
                if phenotype != phenomut:
                    probfold[newmutation] *=(1-probgmut)
                else: continue
        for prob in probfold.values():
            evgndictp+=(1-prob)
        evgndict[seq]+=evgndictp*probg
            
    del neighbourvsphvsprob
    return evgndict

In [ ]:
#phenotypic evolvability
def evolvabilitypND(NDGPmap): 

    evolp = defaultdict(float)
    probgprime = defaultdict(lambda:1) 
    for seq, problist in NDGPmap.items(): 
        phvsprobseq = extractnormalisedprobs(problist,L) 
        neighbourvsphvsprob = {newmutation: extractnormalisedprobs(NDGPmap[newmutation],L) for newmutation in mutationalneighbours(seq)}
        for phenotype, probg in phvsprobseq.items():
            for newmutation, phvsprobmut in neighbourvsphvsprob.items():
               for phenotypeprime, probggprime in phvsprobmut.items():
                    if phenotypeprime != phenotype:
                        probgprime[(phenotype, phenotypeprime)] *=(1-float(probggprime*probg)) 
        del neighbourvsphvsprob
    for (phenotype, phenotypeprime), prob in probgprime.items():
        evolp[phenotype] += (1-prob)
           
    return evolp

## Plastogenetic Congruence

In [ ]:
#create dictionary of mfe probabilities for each genotype
def plastocong(dictRNA12,L):
    pmfedict = defaultdict(float)
    for seq,probs in dictRNA12.items():
        pboltz = extractnormalisedprobs(probs,L)
        pboltzlist = list(pboltz.values())
        mfep = max(pboltzlist)
        #mfepi = pboltzlist.index(mfep)
        pmfedict[seq] = mfep 
    return pmfedict